In [1]:
import torch
import torchtext.vocab as vocab
vocab.pretrained_aliases.keys()

dict_keys(['charngram.100d', 'fasttext.en.300d', 'fasttext.simple.300d', 'glove.42B.300d', 'glove.840B.300d', 'glove.twitter.27B.25d', 'glove.twitter.27B.50d', 'glove.twitter.27B.100d', 'glove.twitter.27B.200d', 'glove.6B.50d', 'glove.6B.100d', 'glove.6B.200d', 'glove.6B.300d'])

In [2]:
[key for key in vocab.pretrained_aliases.keys() if 'glove' in key]

['glove.42B.300d',
 'glove.840B.300d',
 'glove.twitter.27B.25d',
 'glove.twitter.27B.50d',
 'glove.twitter.27B.100d',
 'glove.twitter.27B.200d',
 'glove.6B.50d',
 'glove.6B.100d',
 'glove.6B.200d',
 'glove.6B.300d']

In [6]:
cache_dir = '../glove'
glove = vocab.GloVe(name='6B',dim=50,cache=cache_dir)
'''
stoi: 词到索引的字典：
itos: 一个列表，索引到词的映射；
vectors: 词向量。
'''


100%|█████████▉| 399999/400000 [00:05<00:00, 74832.11it/s]


'\nstoi: 词到索引的字典：\nitos: 一个列表，索引到词的映射；\nvectors: 词向量。\n'

In [7]:
#用knn逻辑求近义词
def knn(W, x, k):
    # 添加的1e-9是为了数值稳定性
    cos = torch.matmul(W, x.view((-1,))) / (
        (torch.sum(W * W, dim=1) + 1e-9).sqrt() * torch.sum(x * x).sqrt())
    _, topk = torch.topk(cos, k=k)
    topk = topk.cpu().numpy()
    return topk, [cos[i].item() for i in topk]

In [8]:
def get_similar_tokens(query_token, k, embed):
    topk, cos = knn(embed.vectors,
                    embed.vectors[embed.stoi[query_token]], k+1)
    for i, c in zip(topk[1:], cos[1:]):  # 除去输入词
        print('cosine sim=%.3f: %s' % (c, (embed.itos[i])))

In [9]:
#除了求近义词以外，我们还可以使用预训练词向量求词与词之间的类比关系。例如，“man”（男人）: “woman”（女人）:: “son”（儿子） : “daughter”（女儿）是一个类比例子：“man”之于“woman”相当于“son”之于“daughter”。求类比词问题可以定义为：对于类比关系中的4个词 a:b::c:da:b::c:d，给定前3个词aa、bb和cc，求dd。设词ww的词向量为vec(w)vec(w)。求类比词的思路是，搜索与vec(c)+vec(b)−vec(a)vec(c)+vec(b)−vec(a)的结果向量最相似的词向量。

In [10]:
def get_analogy(token_a, token_b, token_c, embed):
    vecs = [embed.vectors[embed.stoi[t]]
                for t in [token_a, token_b, token_c]]
    x = vecs[1] - vecs[0] + vecs[2]
    topk, cos = knn(embed.vectors, x, 1)
    return embed.itos[topk[0]]